# EE2211 Tutorial 10



## @author: KAToh


## Question 3 Python Codes

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 18 17:08:04 2020
@author: KAToh
"""
import numpy as np
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## load data set
def load_data(Train=False):
    import csv
    data = []
    ## Read the training data
    f = open('spambase.data')
    reader = csv.reader(f)
    next(reader, None)
    for row in reader:
        data.append(row)
    f.close()
    ## x[:-1]: omit the last element of each x row
    X = np.array([x[:-1] for x in data]).astype(np.float)
    ## x[-1]: the first element from the right instead of from the left 
    y = np.array([x[-1] for x in data]).astype(np.float)
    del data # free up the memory
    if Train:
        # returns X_train, X_test, y_train, y_test
        return train_test_split(X, y, test_size=0.2, random_state=8)
    else:
        return X, y

    
## Get training and test sets   
X_train, X_test, y_train, y_test = load_data(Train=True)

print (X_train.shape)
print (X_test.shape)
print (y_train)

## Linear regression
inv_XTX = np.linalg.inv(X_train.transpose().dot(X_train))
pinv = inv_XTX.dot(X_train.transpose())
W = pinv.dot(y_train)
## Prediction
y_predict = X_test.dot(W)

print(y_predict)

## Calculate classification error rate
yp_cls = [1 if yout >= 0 else 0 for yout in y_predict]   
difference = np.abs(y_test - yp_cls)
test_error_count = (difference == 1).sum()
test_error_rate = test_error_count/len(y_test)
print(test_error_rate)

##--- Compute FPR and FNR at different thresholds ---##
## separate the two classes of predicted data based on the ground truth y_test
pos_idx = np.where(y_test == 1) # identify the indexing of positive-class in the test set
neg_idx = np.where(y_test == 0) # identify the indexing of negative-class in the test set
y_predict_for_PosSamples = y_predict[pos_idx] # prediction of the positive-class data
y_predict_for_NegSamples = y_predict[neg_idx] # prediction of the negative-class data
## use the shorter among the two arrays as threshold
if ( len(y_predict_for_PosSamples) <= len(y_predict_for_NegSamples) ): 
    sorted = np.sort(y_predict_for_PosSamples) # sort in ascending order to be used as threshold
else:
    sorted = np.sort(y_predict_for_NegSamples) # sort in ascending order to be used as threshold
FNR = []
FPR = []  
TPR = []  
## Compute FNR, FPR, and TPR for each threshold
for k in range(len(sorted)):
    yp_cls_pos = np.abs([1 if yout >= sorted[k] else 0 for yout in y_predict_for_PosSamples])   
    yp_cls_neg = np.abs([1 if yout >= sorted[k] else 0 for yout in y_predict_for_NegSamples]) 
    FNR += [(yp_cls_pos == 0).sum()/len(y_predict_for_PosSamples)]
    FPR += [(yp_cls_neg == 1).sum()/len(y_predict_for_NegSamples)]
    TPR += [1-(yp_cls_pos == 0).sum()/len(y_predict_for_PosSamples)]
##--- Plot ROC and DET curves ---##
plt.plot(FPR, FNR, '-', label = 'DET')
plt.plot(FPR, TPR, '-', label = 'ROC')
plt.xlabel('FPR') 
plt.ylabel('FNR/TPR')
plt.legend(fontsize=15)
##--- Comput AUC ---##
ypos_array = [[1 if y_predict_for_PosSamples[j] >= y_predict_for_NegSamples[k] else 0
               for j in range(len(y_predict_for_PosSamples))] for k in range(len(y_predict_for_NegSamples))]    
AUC = np.sum(ypos_array)/(len(y_predict_for_PosSamples)*len(y_predict_for_NegSamples))
print(AUC)

ypos_array_1=[[0 for i in range(len(y_predict_for_PosSamples))] for j in range(len(y_predict_for_NegSamples))]

for j in range(len(y_predict_for_PosSamples)) :
    for k in range(len(y_predict_for_NegSamples)) :
        if y_predict_for_PosSamples[j] >= y_predict_for_NegSamples[k] :
            ypos_array_1[k][j]= 1
        else :
            ypos_array_1[k][j]= 0


FileNotFoundError: [Errno 2] No such file or directory: 'spambase.data'